# The microservice in Python to run the reaction SMARTS on reactants supplied in SMILES

In [ ]:
#loading the packages
from rdkit import Chem
from rdkit import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdChemReactions
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from itertools import chain

In [ ]:
#data input from command line
smiles_reactant = input('Enter molecule into the reaction in SMILES format: ')
molecule = []
molecule.append(smiles_reactant)

#checking if the SMILES code and the molecules are ok
m = Chem.MolFromSmiles(smiles_reactant,sanitize=False)
if m is None:
    print('Invalid! For more information check: https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system ')
else:
    print('\n The SMILES code is OK! We can proceed :)\n')

z = Chem.MolFromSmiles(smiles_reactant)
if z is None:
  print('invalid! For more information check: https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system ')
else:
    print('\n The molecule is chemically reasonable! We can proceed :) \n')


mols = [Chem.MolFromSmiles(smi) for smi in molecule]

print('For detailed information about SMARTS format check: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html')
reaction_smarts = input('\nPlease provide the reaction in SMARTS format: ')

# Convert the SMILES reactant string to a molecule object
reactant_mol = AllChem.MolFromSmiles(smiles_reactant)

# Convert the SMARTS reaction string to a reaction object
reaction = AllChem.ReactionFromSmarts(reaction_smarts)

# Get the reactant and product parts of the reaction
reactant_smarts, product_smarts = AllChem.ReactionToSmarts(reaction).split(">>")

# Find the matches between the reactant in the SMILES format and the reactant from the SMARTS reaction
matches = reactant_mol.GetSubstructMatches(Chem.MolFromSmarts(reactant_smarts))
# Check if there are any matches between the reactant in the SMILES format and the reactant from the SMARTS reaction
if len(matches) > 0:
    print("Reactant in SMILES format is consistent with reactant from SMARTS reaction.")
else:
    print("Reactant in SMILES format is not consistent with reactant from SMARTS reaction.")
    

# helper functions to avoid typing the same thing over and over
def to_smiles(mol_tuple):
    return tuple(Chem.MolToSmiles(mol) for mol in mol_tuple)

def from_smiles(smiles_tuple):
    return tuple(Chem.MolFromSmiles(smiles) for smiles in smiles_tuple)
reaction = AllChem.ReactionFromSmarts(reaction_smarts)

    
all_products_tuples = [reaction.RunReactants((mol, )) for mol in mols]
all_products = chain.from_iterable(all_products_tuples)

# convert to smiles and back to keep only unique products
all_products_smiles = [to_smiles(mol_tuple) for mol_tuple in all_products]
all_products_unique = [from_smiles(smiles_tuple) for smiles_tuple in set(all_products_smiles)]
nProductSets = len(all_products_unique)
print("\nNumber of unique product sets:",nProductSets)

# draw each product set on its own line
prod_tpl = tuple(chain(*all_products_unique))
Draw.MolsToGridImage(prod_tpl,maxMols=10,molsPerRow=reaction.GetNumProductTemplates())